In [15]:
import pandas as pd
import torch.nn as nn
import torch
df = pd.read_csv('./data.csv')
df_eval = pd.read_csv('./evaluation_data.csv')

In [2]:
device = torch.device("cuda") 

In [16]:
df.head()

,instant,dteday,season,yr,mnth,hr,holiday,weekday,workingday,weathersit,temp,atemp,hum,windspeed,casual,registered,cnt
0,1,2011-01-01,1,0,1,0,0,6,0,1,0.24,0.2879,0.81,0.0,3,13,16
1,2,2011-01-01,1,0,1,1,0,6,0,1,0.22,0.2727,0.80,0.0,8,32,40
2,3,2011-01-01,1,0,1,2,0,6,0,1,0.22,0.2727,0.80,0.0,5,27,32
3,4,2011-01-01,1,0,1,3,0,6,0,1,0.24,0.2879,0.75,0.0,3,10,13
4,5,2011-01-01,1,0,1,4,0,6,0,1,0.24,0.2879,0.75,0.0,0,1,1


In [17]:
train = df.sample(frac=0.8,random_state=200) #random state is a seed value
valid = df.drop(train.index)
test = df_eval

X_train = train.drop(columns = ['casual','registered', 'cnt', 'instant', 'dteday'])
Y_train = train['cnt']

X_valid = valid.drop(columns = ['casual','registered', 'cnt', 'instant', 'dteday'])
Y_valid= valid['cnt']

X_test = test.drop(columns = ['dteday'])
X_test.head()


,season,yr,mnth,hr,holiday,weekday,workingday,weathersit,temp,atemp,hum,windspeed
0,1,0,1,0,0,4,1,1,0.26,0.2273,0.56,0.3881
1,1,0,1,1,0,4,1,1,0.26,0.2727,0.56,0.0000
2,1,0,1,2,0,4,1,1,0.26,0.2727,0.56,0.0000
3,1,0,1,3,0,4,1,1,0.26,0.2576,0.56,0.1642
4,1,0,1,4,0,4,1,1,0.26,0.2576,0.56,0.1642


In [18]:
class SimpleClassifier(nn.Module):

    def __init__(self, num_inputs, num_h1, num_h2, num_outputs):
        super().__init__()
        # Initialize the modules we need to build the network
        self.act_fn = nn.ReLU()
        
        self.linear1 = nn.Linear(num_inputs, num_h1)
        self.hidden1 = nn.Linear(num_h1, num_h2)
        self.linear2 = nn.Linear(num_h2, num_outputs)

    def forward(self, x):
        # Perform the calculation of the model to determine the prediction
        x = self.linear1(x)
        x = self.act_fn(x)
        x = self.hidden1(x)
        x = self.act_fn(x)
        x = self.linear2(x)
        return x

In [19]:
import torch.utils.data as data
import torch.optim as optim

In [20]:
X_train.head()

,season,yr,mnth,hr,holiday,weekday,workingday,weathersit,temp,atemp,hum,windspeed
4396,4,0,10,5,0,6,0,1,0.42,0.4242,0.71,0.1642
6205,1,1,2,19,0,2,1,2,0.36,0.3485,0.50,0.1642
2558,2,0,6,12,0,2,1,1,0.64,0.6212,0.41,0.3582
7225,2,1,4,10,0,4,1,1,0.48,0.4697,0.63,0.0896
3305,3,0,8,15,0,0,0,3,0.72,0.6970,0.74,0.2985


In [21]:
import torch
import numpy as np

train_dataset = data.TensorDataset(torch.from_numpy(np.array(X_train)), torch.from_numpy(np.array(Y_train)))
valid_dataset = data.TensorDataset(torch.from_numpy(np.array(X_valid)), torch.from_numpy(np.array(Y_valid)))

train_data_loader = data.DataLoader(train_dataset, batch_size=128, shuffle=True)

In [23]:
model = SimpleClassifier(12, 500, 500, 1).to(device)
MSELoss = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=0.01)

model.train()

for epoch in range(100):  # loop over the dataset multiple times
    
    sum_loss = 0

    for inputs, labels in train_data_loader:
        # get the inputs; data is a list of [inputs, labels]

        inputs, labels = inputs.to(device), labels.to(device)

        # zero the parameter gradients
        optimizer.zero_grad()

        # forward + backward + optimize
        preds = model(inputs.float())
        preds = preds.squeeze(dim=1)
        loss = MSELoss(preds, labels.float())
        
        loss.backward()
        optimizer.step()

        sum_loss += loss
    
    print(epoch, sum_loss / X_train.shape[0])


0 tensor(208.5496, grad_fn=<DivBackward0>)
1 tensor(131.0928, grad_fn=<DivBackward0>)
2 tensor(118.6046, grad_fn=<DivBackward0>)
3 tensor(114.1174, grad_fn=<DivBackward0>)
4 tensor(107.3128, grad_fn=<DivBackward0>)
5 tensor(98.7039, grad_fn=<DivBackward0>)
6 tensor(90.8969, grad_fn=<DivBackward0>)
7 tensor(91.0296, grad_fn=<DivBackward0>)
8 tensor(78.0611, grad_fn=<DivBackward0>)
9 tensor(76.8497, grad_fn=<DivBackward0>)
10 tensor(68.7771, grad_fn=<DivBackward0>)
11 tensor(70.1313, grad_fn=<DivBackward0>)
12 tensor(67.0685, grad_fn=<DivBackward0>)
13 tensor(61.2939, grad_fn=<DivBackward0>)
14 tensor(61.4552, grad_fn=<DivBackward0>)
15 tensor(53.4853, grad_fn=<DivBackward0>)
16 tensor(55.7141, grad_fn=<DivBackward0>)
17 tensor(51.9816, grad_fn=<DivBackward0>)
18 tensor(49.1597, grad_fn=<DivBackward0>)
19 tensor(54.9103, grad_fn=<DivBackward0>)
20 tensor(100.5306, grad_fn=<DivBackward0>)
21 tensor(61.3941, grad_fn=<DivBackward0>)
22 tensor(58.2959, grad_fn=<DivBackward0>)
23 tensor(50.47

In [24]:
X_test.dtypes

season          int64
yr              int64
mnth            int64
hr              int64
holiday         int64
weekday         int64
workingday      int64
weathersit      int64
temp          float64
atemp         float64
hum           float64
windspeed     float64
dtype: object

In [25]:
all_preds = []
with torch.no_grad(): # Deactivate gradients for the following code
    for data_inputs in X_test.values:
        preds = model(torch.tensor(data_inputs).float().to(device))
        preds = preds.squeeze(dim=0)
        all_preds.append(preds.cpu())

pd.DataFrame(all_preds).to_csv("results.csv", index=False, header=None)
        

In [26]:
def rmsle(y_true,y_pred):
    n = len(y_true)
    msle = np.mean([(np.log(max(y_pred[i],0) + 1) - np.log(y_true[i] + 1)) ** 2.0 for i in range(n)])
    return np.sqrt(msle)

In [27]:
all_inputs = []
all_labels = []

with torch.no_grad():
    for inputs, labels in valid_dataset:
        preds = model(inputs.float().to(device))
        all_inputs.append(float(preds))
        all_labels.append(float(labels))

print(rmsle(np.array(all_labels), np.array(all_inputs)))

0.5822698513762321


: 